In [162]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

df = pd.read_csv("student_data.csv")



In [163]:
# df.columns

In [164]:
# df.head()

In [165]:
# df.info()

In [166]:
# df.keys()

In [167]:
df['alcohol_level'] = (df['Dalc'] + df['Walc']) / 2
df['parent_education_avg'] = (df['Medu'] + df['Fedu']) / 2
df['social_activity'] = (df['freetime'] + df['goout']) / 2
df["academic_risk"] = df["failures"] / (df["studytime"] + 1)

df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,Walc,health,absences,G1,G2,G3,alcohol_level,parent_education_avg,social_activity,academic_risk
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,1,3,6,5,6,6,1.0,4.0,3.5,0.0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,1,3,4,5,5,6,1.0,1.0,3.0,0.0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,3,10,7,8,10,2.5,1.0,2.5,1.0
3,GP,F,15,U,GT3,T,4,2,health,services,...,1,5,2,15,14,15,1.0,3.0,2.0,0.0
4,GP,F,16,U,GT3,T,3,3,other,other,...,2,5,4,6,10,10,1.5,3.0,2.5,0.0


In [168]:
X = df.drop(columns=[
    "G1", "G2", "G3", "school", "famrel", "internet",
    "address", "Pstatus", "guardian", "reason",
    "romantic", "nursery", "Dalc", "Walc", "Medu", 
    "Fedu", "freetime", "goout", "failures", "studytime",
    "sex", "famsize", "Mjob", "Fjob", "schoolsup", "famsup",
    
 ])

y = (df["G3"] >= 10).astype(int)
X

,age,traveltime,paid,activities,higher,health,absences,alcohol_level,parent_education_avg,social_activity,academic_risk
0,18,2,no,no,yes,3,6,1.0,4.0,3.5,0.000000
1,17,1,no,no,yes,3,4,1.0,1.0,3.0,0.000000
2,15,1,yes,no,yes,3,10,2.5,1.0,2.5,1.000000
3,15,1,yes,yes,yes,5,2,1.0,3.0,2.0,0.000000
4,16,1,yes,no,yes,5,4,1.5,3.0,2.5,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
390,20,1,yes,no,yes,4,11,4.5,2.0,4.5,0.666667
391,17,2,no,no,yes,2,3,3.5,2.0,4.5,0.000000
392,21,1,no,no,yes,3,3,3.0,1.0,4.0,1.500000
393,18,3,no,no,yes,5,0,3.5,2.5,2.5,0.000000


In [169]:
X.head()


,age,traveltime,paid,activities,higher,health,absences,alcohol_level,parent_education_avg,social_activity,academic_risk
0,18,2,no,no,yes,3,6,1.0,4.0,3.5,0.0
1,17,1,no,no,yes,3,4,1.0,1.0,3.0,0.0
2,15,1,yes,no,yes,3,10,2.5,1.0,2.5,1.0
3,15,1,yes,yes,yes,5,2,1.0,3.0,2.0,0.0
4,16,1,yes,no,yes,5,4,1.5,3.0,2.5,0.0


In [170]:
numeric_features = [
    "age",
    "traveltime",
    "academic_risk",
    "health",
    "absences",
    "alcohol_level",
    "parent_education_avg",
    "social_activity"
]

categorical_features = [
    "paid",
    "activities",
    "higher",
]


In [171]:
###  new


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(
        n_estimators=200,
        random_state=42
    ))
])


In [172]:
# from sklearn.preprocessing import StandardScaler, OneHotEncoder

# numeric_transformer=Pipeline(steps=[
#     ("scaler", StandardScaler())
# ])

# categorical_transformer = Pipeline(steps=[
#     ("onehot", OneHotEncoder(handle_unknown="ignore"))
# ])

In [173]:
# X=pd.get_dummies(X, columns=categorical_features, drop_first=True)

# scaler=StandardScaler()
# X[numeric_features]=scaler.fit_transform(X[numeric_features])

In [174]:
# X.all()

In [175]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))


Accuracy: 0.7341772151898734


In [176]:
import joblib
joblib.dump(model, "student_model.pkl")


['student_model.pkl']

In [177]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

lr_pred = lr.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)

print("Logistic Regression Accuracy:", lr_acc)


ValueError: could not convert string to float: 'yes'

In [ ]:
rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)
rf_acc = accuracy_score(y_test, rf_pred)


print("Random Forest Accuracy:", rf_acc)


Random Forest Accuracy: 0.759493670886076


In [ ]:
svm = SVC(kernel="rbf", probability=True)
svm.fit(X_train, y_train)

svm_pred = svm.predict(X_test)
svm_acc = accuracy_score(y_test, svm_pred)

print("SVM Accuracy:", svm_acc)


SVM Accuracy: 0.7341772151898734


In [ ]:
results = pd.DataFrame({
    "Model": ["Logistic Regression", "Random Forest", "SVM"],
    "Accuracy": [lr_acc, rf_acc, svm_acc]
})

print(results.sort_values(by="Accuracy", ascending=False))


                 Model  Accuracy
1        Random Forest  0.759494
0  Logistic Regression  0.746835
2                  SVM  0.734177


In [ ]:
# # bu yerda grafik kodi bor
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

# cm = confusion_matrix(y_test, rf_pred)

# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
# plt.xlabel("Predicted")
# plt.ylabel("Actual")
# plt.title("Confusion Matrix - Random Forest")
# plt.show()

In [ ]:
import joblib

joblib.dump(rf, "rf_model.pkl")    # eng yaxshi model
joblib.dump(lr, "lr_model.pkl") 
joblib.dump(svm, "svm_model.pkl")
# joblib.dump(scaler, "scaler.pkl")


['svm_model.pkl']

In [ ]:
# sample_student = {
#     "age": 18,                   # Talabaning yoshi
#     "traveltime": 4,             # Maktabga borish vaqti (1 = juda yaqin, 4 = uzoq)
#     "health": 1,                 # Sog‘liq darajasi (1 = yomon, 5 = yaxshi)
#     "absences": 15,               # Qoldirilgan darslar soni
#     "alcohol_level": 1,          # Spirtli ichimlik darajasi (0 = ichmaydi)
#     "parent_education_avg": 2,   # Ota-ona ta’lim darajasi o‘rtachasi (1–5)
#     "social_activity": 4,        # Ijtimoiy faollik darajasi (1 = kam, 5 = ko‘p)
#     "academic_risk": 2,        # Akademik xavf (yuqori bo‘lsa — xavf katta)

#     "paid": "no",               # Qo‘shimcha pullik darslar (yes/no)
#     "activities": "no",         # Darsdan tashqari faoliyat (yes/no)
#     "higher": "no",             # Oliy ta’lim olish istagi (yes/no)
# }


# sample_df = pd.DataFrame([sample_student])
# sample_df = pd.get_dummies(sample_df)



# # model_features = joblib.load("model_features.pkl")
# # sample_df = sample_df.reindex(columns=model_features, fill_value=0)
# # sample_df[numeric_features] = scaler.transform(sample_df[numeric_features])

# model = joblib.load("rf_model.pkl")

# prediction = model.predict(sample_df)
# confidence = model.predict_proba(sample_df)[0][prediction[0]]



# print("Result:", "Talaba o'tdi :) ✅ " 
# if prediction[0] == 1
# else "Talaba o'ta olmadi :( ❌ ")
# print("Ishonchlilik:", round(confidence * 100, 2), "%")


In [ ]:
import joblib

scaler = joblib.load("scaler.pkl")
print(scaler.feature_names_in_)


['age' 'traveltime' 'academic_risk' 'health' 'absences' 'alcohol_level'
 'parent_education_avg' 'social_activity']


In [178]:
sample_df = pd.DataFrame([{
    "age": 18,
    "traveltime": 4,
    "health": 1,
    "absences": 15,
    "alcohol_level": 1,
    "parent_education_avg": 2,
    "social_activity": 4,
    "academic_risk": 2,
    "paid": "no",
    "activities": "no",
    "higher": "no"
}])

pred = model.predict(sample_df)[0]
proba = model.predict_proba(sample_df)[0][pred]

print("Natija:", "O‘TDI" if pred else "O‘TMADI")
print("Ishonchlilik:", round(proba*100, 2), "%")


Natija: O‘TMADI
Ishonchlilik: 72.5 %
